

1.   Ниже представлено решение задачи на информационный поиск. Ссылка на датасет https://www.kaggle.com/sarahdejong/lyrics (>300мб)
2.   Он представляет собой набор, популярных песен на английском, выложенных в сеть с 2006 года. В его входит, название, атвор, текст а также неоторые другие параметры
3.   Была произведена попытка создания модели, позволяющей по набору слов(запросу) сапоставления названием песен и их содержанию











In [45]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request
import re
import nltk
from gensim.models import Word2Vec
from sklearn import preprocessing

nltk.download('punkt')
nltk.download('stopwords')


lyrics = pd.read_csv('/content/drive/MyDrive/lyrics.csv', sep=',', index_col='index')
with open('/content/drive/MyDrive/popsongs.txt', 'r', encoding='utf-8') as g:
    article_text = g.read()
lyrics.head(10)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,song,year,artist,genre,lyrics
index,,,,,
0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."
5,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard\nChurch bells ringing\nI heard\nA choi...
6,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend\nW...
7,waiting,2009,beyonce-knowles,Pop,"Waiting, waiting, waiting, waiting\nWaiting, w..."
8,slow-love,2009,beyonce-knowles,Pop,[Verse 1:]\nI read all of the magazines\nwhile...


Я решил воспользоватьтся методом Word2vec, потому что, на мой взгляд он достаточно релевантен в данной задаче, прежде всего я составил словарь из слов песен по средствам данного метода и использовал для этого текст всех песен

In [46]:
# Preparing the dataset
processed_article = article_text.replace('/n', '')
processed_article = processed_article.lower()
processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )
processed_article = re.sub(r'\s+', ' ', processed_article)

all_sentences = nltk.sent_tokenize(processed_article)
all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

In [47]:
# Removing Stop Words
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]

In [ ]:
word2vec = Word2Vec(all_words, min_count=2)
vocabulary = list(word2vec.wv.vocab)
print(len(vocabulary))

25228


В словарь я определил те слова, которые встречались более чем 2 раза в совокупности всех текстов, однако из них были исключены stopwords и прочее, мешающие анализу в будущем, также я пришел к выводу, что обучение модели на основе датсета песен могло быть не лучшей идеей, так как она могла давать довольно четкие предсказания сходств слов 'man', 'boy', но не показывать сходства в более банальных случаях. Решение этого сводилось бы к использования созданного набор векторов схожести для слов от Word2vec, либо обучение на стороннем датасете

In [ ]:
#trash
def text_listing(text):
  text = text.replace('/n', '')
  text = text.lower()
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  text = text.split(' ')
  
my_answer = 'Stressed russia'
my_list = text_listing(my_answer)
print(my_list)

name_list =[]
for i in my_list:
  for j in name_list:
    pass

word2vec.wv.similarity('man', 'boy')



['stressed', 'russia']


0.7641143

Также я решил использовать метод обратного индексирования, который заключается в сопоставления слов текстам, в которых они встречаются, это обеспечивает снижения набора текстов, необходимых для сравнения с запросом. 


Данную функцию я реализовал как пересечение множеств в которых втсречается индексы, при условии ими выдачи колличества запросов не более указанного (зависит от изначальной феличины датасета и необходимой степени сжатия), а также измерении объема списка текстов с представленными индексом и возврат их по мере увеличения встречаемости слов в данных текстах  

In [ ]:
def indexing(list_index, df):
    dict_index = dict.fromkeys(list_index)
    for i in list_index:
      for j in df.lyrics:
        if type(j) != float:
          text_listing(j) 
          if i in j: 
            if dict_index[i]==None: dict_index[i] = j
            else: dict_index[i] = dict_index[i].append(j)
    return dict_index


dict_index = indexing(vocabulary,lyrics)


In [52]:
def indexing_intersect(query):
  query = text_listing(query)
  query_index = set(dict_index)
  for i in query: query_index = (query_index & indexing[i])
  if len(query_index) >= 10000:
    query_index = set(dict_index)
    i_list = []
    for i in query: 
      i_list = i_list.append([len(indexing[i]), indexing[i]])
    i_list = i_list.sort(reverse=True)
    for i in i_list:
      while query_index & i_list[1]<=10000:
        query_index = query_index & i_list[1]
  return list(query_index)



Я в процессе решения задачи использовал разные метрики, в том числе euclidean_similarity для векторов слов запроса и слов текстов, где выводилось:



**mean(max(euclidean_similarity(x(i), y(j), j in lyrics.shape[0]), i in range(len(index_dictionary))))**


После чего происходило сравнение данной метрики для проиндексированных текстов

In [ ]:
# def euclidean_distance(x, y):
#     x = preprocessing.normalize([x])[0]
#     y = preprocessing.normalize([y])[0]
#     return sum((x[i] - y[i]) ** 2 for i in range(len(x))) ** .5


# def euclidean_similarity(x, y):
#     return (1/(1 + euclidean_distance(x, y)))

Но рабочий вариант остановился на метрике схожести между словами в пакете Word2vec

In [53]:
import random

class Document:
    def __init__(self, title, text):
        # можете здесь какие-нибудь свои поля подобавлять
        self.title = title
        self.text = text
    
    def format(self, query):
        # возвращает пару тайтл-текст, отформатированную под запрос
        return [self.title, self.text + ' ...']

my_index=[]
def build_index_new():
    my_index = []
    for i in range(lyrics.shape[0]):
        my_index.append(Document(lyrics.song[i], lyrics.lyrics[i]))
    return(my_index)

def score(query, document):
    score_list = [0,0]
    for i in list(dict(word2vec.wv.most_similar(query,topn=100)).keys()):
        if i.lower() in " ".join(document.format(query)):
            score_list[0] += 1
            score_list[1] += dict(word2vec.wv.most_similar(query,topn=100)).get(i) 
    return score_list[1]/score_list[0]

def retrieve(query, pot_index):
    # возвращает начальный список релевантных документов
    # (желательно, не бесконечный)
    candidates = []
    for doc in my_index:
        if query.lower() in doc.title.lower() or query in doc.text.lower():
            candidates.append(doc)
    return candidates[:50]


Была использована Mean Average Precision определяющая объем акутальных песен из вычисленных предсказаний для данного запроса, которая была использована для опеделения качества модели:

In [ ]:
def apk(actual, predicted, k=50):
    if not actual: return 0
    predicted = predicted[:k]
    result = 0
    result_num = 0
    for i,j in enumerate(predicted):
        if j in actual and j not in predicted[:i]:
            result_num += 1
            result += result_num / (i+1)
    return result / min(len(actual), k)


def mapk(actual, predicted, k=50):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
lyrics.shape[0]

362237

Также была идея добавление среднего схожести между значимостью схожести слов между текстом и названиями и добавлением условия на авторство, если в запросе указан автор, то приоритетным становится показ именно данного автора, но представленное сопоставление песен и певцов требовало расширение певцов, за счет наличие там групп, разных псевдонимов и прочего, что привело бы к использованию сторонних данных 


In [ ]:
from flask import Flask, render_template, request
from time import time

app = Flask(__name__, template_folder='.')
my_index = build_index_new()


@app.route('/', methods=['GET'])
def index(my_index):
    start_time = time()
    query = request.args.get('query')
    if query is None:
        query = ''
    my_index = my_index[indexing_intersect(query)]
    documents = retrieve(query, my_index)
    documents = sorted(documents, key=lambda doc: -score(query, doc))
    results = [doc.format(query)+['%.2f' % score(query, doc)] for doc in documents] 
    return render_template(
        '/content/drive/MyDrive/index.html',
        time="%.2f" % (time()-start_time),
        query=query,
        search_engine_name='Neptun Compiuter',
        results=results
    )


if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=1228)
